In [1]:
#pandas version

import pandas as pd
pd.__version__

'2.2.3'

In [5]:
#libraries
import pandas as pd
import re

In [20]:
#task 1
urls = [
    "https://en.wikipedia.org/wiki/List_of_earthquakes_2001%E2%80%932010",
    "https://en.wikipedia.org/wiki/List_of_earthquakes_2011%E2%80%932020",
    "https://en.wikipedia.org/wiki/List_of_earthquakes_2021%E2%80%932030"]

all_tables = []

for url in urls:
    tables = pd.read_html(url, header=0) #reading the tables
    for tbl in tables:
        if tbl.shape[1] >= 8: #keeping tables with 8+ columns
            all_tables.append(tbl)

eq_df = pd.concat(all_tables, ignore_index=True)

eq_df.head(5)

,Date,Time (UTC),Place,Lat.,Long.,Fatalities,Magnitude,Comments,Sources,Source
0,"January 1, 2001",06:57,"Mindanao, Philippines",6.898,126.579,0,7.5,Mw (HRV).,NaN,NaN
1,"January 9, 2001",16:49,Vanuatu,-14.928,167.170,0,7.1,Mw (USGS).,NaN,NaN
2,"January 10, 2001",16:02,"Kodiak Island region, Alaska, United States",57.078,-153.211,0,6.9,Mw (HRV).,NaN,NaN
3,"January 13, 2001",17:33,Offshore El Salvador see January 2001 El Salv...,13.049,-88.660,1145,7.7,Mw (HRV).,NaN,NaN
4,"January 26, 2001",03:16,"Gujarat, India see 2001 Gujarat earthquake",23.419,70.232,20023,7.7,Mw (HRV).,NaN,NaN


In [ ]:
#task 2: data cleaning

#part 1: making combined datetime column
eq_df['datetime'] = pd.to_datetime(
    eq_df['Date'].astype(str) + ' ' + eq_df['Time (UTC)'].astype(str),
    errors='coerce'
)

#part 2: fatalities
def fatalities(x):
    if pd.isna(x):
        return pd.NA

    s = str(x).lower()

    # finding the lower range
    m = re.search(r'(\d[\d,]*)\s*[~–-]\s*\d[\d,]*', s)
    if m:
        return int(m.group(1).replace(',', ''))

    # “dead” or “killed”
    m = re.search(r'(\d[\d,]*)\s*(dead|killed)', s)
    if m:
        return int(m.group(1).replace(',', ''))

    # only missing
    if 'missing' in s and 'dead' not in s and 'killed' not in s:
        m = re.search(r'(\d[\d,]*)', s)
        if m:
            return int(m.group(1).replace(',', ''))

    # first number
    m = re.search(r'(\d[\d,]*)', s)
    if m:
        return int(m.group(1).replace(',', ''))

    # nan
    return pd.NA

eq_df['Fatalities'] = eq_df['Fatalities'].apply(fatalities)

# eq_df[['Fatalities']].head(10)
# eq_df['Fatalities'].dtype
# eq_df.head(5)

,Fatalities
0,0
1,0
2,0
3,1145
4,20023
5,407
6,0
7,1
8,3
9,0
